In [1]:
!pip install langchain
!pip install langchain_openai
!pip install langgraph
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.4/109.4 kB 2.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 12.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 6.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.0/409.0 kB 10.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 32.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 5.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.0/616.0 kB 13.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 3.6 MB/s 

In [56]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.prebuilt import create_react_agent
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

import os
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import AzureChatOpenAI


os.environ["AZURE_OPENAI_API_KEY"] = 'b3e819600fbe4981be34ef2aa79943e2'

from langchain_core.tools import tool

@tool
def sendEmail(receiver_email: str, message: str):
  """
  use this to send mail to receiver_email with the body of message
  """
  import smtplib, ssl

  port = 587  # For starttls
  smtp_server = "smtp.gmail.com"
  sender_email = "asachoitw@gmail.com"
  password = "pzzh inaj zzle rpcx"
 

  context = ssl.create_default_context()
  with smtplib.SMTP(smtp_server, port) as server:
      server.ehlo()  # Can be omitted
      server.starttls(context=context)
      server.ehlo()  # Can be omitted
      server.login(sender_email, password)
      server.sendmail(sender_email, receiver_email, message, )


@tool
def getingPolicyInfo(customerId: str):
    """
    "use this tool when you need to get Policy Information, and you must provide the customer ID"
    """
   
    return f"policy found for {customerId} product: set for life, premium: 20000"

@tool
def getCustomerID(name: str, phone: str):
  """get the customerID base on customer name and phone number, both phone and name are mandatory"""
  return "12345"

@tool
def getUserInfo(username: str):
    """get user information base on username"""
    print("searching user:" + username)
    import requests
    r = requests.get("https://dummyjson.com/users/")
    users = r.json()["users"]

    user = [u for u in users if u["firstName"] == username]
    return user


In [57]:
def main():
  tools = [getingPolicyInfo, getCustomerID, sendEmail, getUserInfo]
  username = "Olivia"

  llm = AzureChatOpenAI(
      azure_endpoint="https://ik-oai-eastus-2.openai.azure.com/",
      azure_deployment="gpt-4o",
      openai_api_version="2023-09-01-preview",
  )

  agent_executor = create_react_agent(llm, tools)
  prompt = {"messages": [HumanMessage(content=f"extract the policy information, get user information of {username} and send it to asa.choi@gmail.com")]}

  for chunk in agent_executor.stream(prompt):
      print(chunk)
      if(chunk.get("tools")):
        m = chunk["tools"]["messages"][0]
        print(f"{m.name}, {m.content}")

#sendEmail("asa.choi@gmail.com", "how are you?")
main();

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_GAcYDIk9d3f6nna4jWHmboKa', 'function': {'arguments': '{"username": "Olivia"}', 'name': 'getUserInfo'}, 'type': 'function'}, {'id': 'call_xJqVlA210RRIauu1422wwBL6', 'function': {'arguments': '{"name": "Olivia", "phone": "unknown"}', 'name': 'getCustomerID'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 173, 'total_tokens': 225}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_abc28019ad', 'finish_reason': 'tool_calls', 'logprobs': None, 'content_filter_results': {}}, id='run-c18d864e-9626-4c58-9fbe-32f2b331b9ca-0', tool_calls=[{'name': 'getUserInfo', 'args': {'username': 'Olivia'}, 'id': 'call_GAcYDIk9d3f6nna4jWHmboKa'}, {'name': 'getCustomerID', 'args': {'name': 'Olivia', 'phone': 'unknown'}, 'id': 'call_xJqVlA210RRIauu1422wwBL6'}], usage_metadata={'input_tokens': 173, 'output_tokens': 52, 'total_tokens': 225})]}}
searching use